In [2]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt



In [3]:
songs = pd.read_csv("/Users/aayushchadha/Downloads/songdata.csv", header=0)
#songs.head()
songs = songs[songs.artist != 'Lata Mangeshkar']
songs.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [51]:
text_corpus = []
for song in songs['text']:
    words = song.lower().split()
    text_corpus.append(words)

    #ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

57618


In [52]:
import time
start_time = time.time()



songs2vec.train(text_corpus)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 298.3136217594147 seconds ---


In [4]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

In [5]:
print(songs2vec['un-right'])
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songs['song_vector'] = songs['text'].apply(songVector)






[ 0.07639486  0.14794612 -0.02950447 -0.11471502  0.01379999 -0.03689257
  0.08070549  0.11641828 -0.06616479 -0.04819403 -0.05529087  0.04415668
  0.01586323 -0.027852   -0.03093797  0.0462577  -0.01044002 -0.19352244
 -0.06914271  0.16508158 -0.11803728  0.02449879 -0.13418569  0.17019983
 -0.08197258 -0.13308808  0.02446113 -0.09515736 -0.03024296  0.01019404
  0.18805157  0.10736922  0.0434821  -0.22622068 -0.05174229 -0.16702813
  0.00199558 -0.08532391 -0.13357763 -0.17946053  0.09619296 -0.03936654
 -0.0347906  -0.06207792 -0.04948123  0.04932227 -0.00637919  0.0424794
  0.14268583  0.04900933]


NameError: name 'train' is not defined

In [6]:
songs.head()
    

,artist,song,link,text,song_vector
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd...","[[0.0256763, 0.309385, -0.169509, -0.0479835, ..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl...","[[-0.0348163, 0.272111, -0.166464, -0.00871774..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...,"[[0.0478249, 0.302333, -0.169563, -0.0242387, ..."
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...,"[[0.057627, 0.295603, -0.160222, -0.0383688, -..."
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...,"[[0.0593327, 0.299534, -0.161537, -0.0415768, ..."


In [19]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)


,artist,song,link,text,song_vector
14849,Oliver,As Long As He Needs Me,/o/oliver/as+long+as+he+needs+me_20608709.html,"As long as he needs me... \nOh, yes, he does ...","[[-0.0315812, 0.335919, -0.196711, -0.0528897,..."
15645,Paul Simon,You're Kind,/p/paul+simon/youre+kind_20105959.html,"You're kind, you're so kind \nYou rescued me ...","[[0.0302546, 0.273594, -0.19587, -0.0314226, -..."
22629,Zornik,We Are Lost,/z/zornik/we+are+lost_20347214.html,I want to climb up in the sky with you \nTo s...,"[[0.0208253, 0.292814, -0.206927, -0.0749131, ..."
25071,Bee Gees,Loose Talk Costs Lives,/b/bee+gees/loose+talk+costs+lives_20015752.html,Loose talk costs lives \nPeople like to play ...,"[[0.0390722, 0.321847, -0.189492, -0.012435, -..."
29979,Dolly Parton,Hillbilly Willy,/d/dolly+parton/hillbilly+willy_20214066.html,Now Little Will was from the hills way back in...,"[[0.138491, 0.291326, -0.160594, -0.0387402, -..."
41226,Lana Del Rey,Summertime Sadness,/l/lana+del+rey/summertime+sadness_20992968.html,Kiss me hard before you go \nSummertime sadne...,"[[0.0128133, 0.266378, -0.129556, -0.011296, -..."
1088,Barbra Streisand,I Had Myself A True Love,/b/barbra+streisand/i+had+myself+a+true+love_2...,I had myself a true love \nA true love who wa...,"[[0.0967401, 0.295728, -0.168346, -0.0520959, ..."
9894,Judas Priest,Love Zone,/j/judas+priest/love+zone_20076191.html,"Been awake all night, can't get no sleep \nI ...","[[0.0717128, 0.2559, -0.225748, -0.0403028, -0..."
1474,Bing Crosby,'Round And 'Round The Christmas Tree,/b/bing+crosby/round+and+round+the+christmas+t...,Round and round the Christmas tree \nPresents...,"[[0.0930563, 0.312121, -0.0810639, -0.0844544,..."
41506,Leo Sayer,The End Of The Game,/l/leo+sayer/the+end+of+the+game_20718521.html,Though he heard her clearly \nShe said it so ...,"[[0.0242706, 0.293231, -0.174866, -0.0643418, ..."


In [22]:
X = np.array(song_vectors).reshape((5761, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=200, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))



[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 5761
[t-SNE] Computed conditional probabilities for sample 2000 / 5761
[t-SNE] Computed conditional probabilities for sample 3000 / 5761
[t-SNE] Computed conditional probabilities for sample 4000 / 5761
[t-SNE] Computed conditional probabilities for sample 5000 / 5761
[t-SNE] Computed conditional probabilities for sample 5761 / 5761
[t-SNE] Mean sigma: 0.052817
[t-SNE] Iteration 25: error = 1.4294568, gradient norm = 0.0097812
[t-SNE] Iteration 50: error = 1.4023627, gradient norm = 0.0064141
[t-SNE] Iteration 75: error = 1.3296787, gradient norm = 0.0015149
[t-SNE] Iteration 100: error = 1.3173788, gradient norm = 0.0012001
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.317379
[t-SNE] Iteration 125: error = 1.2969683, gradient norm = 0.0008987
[t-SNE] Iteration 125: gradient norm 0.000899. Finished.
[t-SNE] Error after 12

In [28]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

In [29]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,artist,song,link,text,song_vector,X,Y
0,Oliver,As Long As He Needs Me,/o/oliver/as+long+as+he+needs+me_20608709.html,"As long as he needs me... \nOh, yes, he does ...","[[-0.0315812, 0.335919, -0.196711, -0.0528897,...",-3.149459,1.857558
1,Paul Simon,You're Kind,/p/paul+simon/youre+kind_20105959.html,"You're kind, you're so kind \nYou rescued me ...","[[0.0302546, 0.273594, -0.19587, -0.0314226, -...",-3.942099,-1.864561
2,Zornik,We Are Lost,/z/zornik/we+are+lost_20347214.html,I want to climb up in the sky with you \nTo s...,"[[0.0208253, 0.292814, -0.206927, -0.0749131, ...",3.385142,-4.797422
3,Bee Gees,Loose Talk Costs Lives,/b/bee+gees/loose+talk+costs+lives_20015752.html,Loose talk costs lives \nPeople like to play ...,"[[0.0390722, 0.321847, -0.189492, -0.012435, -...",3.368369,-5.955466
4,Dolly Parton,Hillbilly Willy,/d/dolly+parton/hillbilly+willy_20214066.html,Now Little Will was from the hills way back in...,"[[0.138491, 0.291326, -0.160594, -0.0387402, -...",0.033931,3.944600


In [41]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['song'],
    mode='markers',
    marker=dict(
        size='7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

py.iplot(data, filename='song_similarity')